# How to plot one large MOC (union) per service

## Import required modules

In [41]:
from elasticsearch import Elasticsearch
from mocpy import MOC
from ipyaladin import Aladin

## Connect to the ElasticSearch server

In [42]:
es = Elasticsearch('http://voparis-elasticsearch.obspm.fr:9200')
indexName = 'moc-index'
target = "mars"

## Get the list of services

In [43]:
query = {
    "bool": {
         "must": [
             {
                "exists" : {
                    "field" : "coverage"
                }
             },
             {
                 "match" : {
                    "target_name" : target
                 }
            }
        ]
    }
}

aggs = {
    "agg-service" : {
        "terms": {
            "field" : "service_title.keyword",
            "size": 100
        },
    }
}


page = es.search(
    index=indexName,
    query=query,
    size=0,
    fields=["*"],
    aggs = aggs
)

services = []

for bucket in page["aggregations"]["agg-service"]["buckets"] :
    print(bucket["key"])
    services.append(bucket["key"])

Mars_craters_dev
CRISM
hst_planeto
hrsc3nd
Planmap
vvex


## For each service, merge all of its MOCs into one for the target

In [44]:
def mergeMocs(mocsList) :
    mocUnion = MOC.new_empty(29)
    for item in mocsList :
        coverageField = item["fields"]["coverage"]
        if type(coverageField) == type([]) :
            for coverage in coverageField :
                moc = MOC.from_str(coverage)
                mocUnion = mocUnion.union(moc)
        else :
            moc = MOC.from_str(coverageField)
            mocUnion = mocUnion.union(moc)
    return mocUnion

In [46]:
size = 2000

data = []
for service in services :
    print("Merging MOCs of ",service,"...")
    query = {
        "bool": {
             "must": [
                 {
                    "exists" : {
                        "field" : "coverage"
                    }
                },
                 {
                     "match" : {
                        "target_name" : target
                     }
                },
                {
                    "term" : {
                         "service_title.keyword" : service
                     } 
                 } 
            ]
        }
    }

    page = es.search(
        index=indexName,
        query=query,
        size=size,
        scroll="2m",
        fields=["coverage"],
    )
    
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']["value"]
    mocUnion = MOC.new_empty(29)

    progress = 0
    increment = (100*size)/page['hits']['total']["value"]
    while (scroll_size > 0):
        print("%.2f %%" % progress)
        page = es.scroll(scroll_id = sid, scroll = '2m')
        # Update the scroll ID
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        mocUnion = mocUnion.union(mergeMocs(page["hits"]["hits"]))
        progress = progress + increment
        
    data.append({"service":service,"coverage":mocUnion})
    print("100.00 %")

print("DONE")

Merging MOCs of  Mars_craters_dev ...
0.00 %
0.52 %
1.04 %
1.56 %
2.08 %
2.60 %
3.12 %
3.64 %
4.16 %
4.68 %
5.20 %
5.72 %
6.24 %
6.76 %
7.28 %
7.80 %
8.32 %
8.84 %
9.36 %
9.88 %
10.40 %
10.92 %
11.44 %
11.96 %
12.48 %
13.00 %
13.52 %
14.04 %
14.56 %
15.08 %
15.60 %
16.12 %
16.64 %
17.16 %
17.68 %
18.20 %
18.72 %
19.24 %
19.76 %
20.28 %
20.80 %
21.32 %
21.84 %
22.36 %
22.88 %
23.40 %
23.92 %
24.44 %
24.96 %
25.48 %
26.00 %
26.52 %
27.04 %
27.56 %
28.08 %
28.60 %
29.12 %
29.64 %
30.16 %
30.68 %
31.20 %
31.72 %
32.24 %
32.76 %
33.28 %
33.80 %
34.32 %
34.84 %
35.37 %
35.89 %
36.41 %
36.93 %
37.45 %
37.97 %
38.49 %
39.01 %
39.53 %
40.05 %
40.57 %
41.09 %
41.61 %
42.13 %
42.65 %
43.17 %
43.69 %
44.21 %
44.73 %
45.25 %
45.77 %
46.29 %
46.81 %
47.33 %
47.85 %
48.37 %
48.89 %
49.41 %
49.93 %
50.45 %
50.97 %
51.49 %
52.01 %
52.53 %
53.05 %
53.57 %
54.09 %
54.61 %
55.13 %
55.65 %
56.17 %
56.69 %
57.21 %
57.73 %
58.25 %
58.77 %
59.29 %
59.81 %
60.33 %
60.85 %
61.37 %
61.89 %
62.41 %
62.93 %
63.45 

In [52]:
aladin = Aladin(
    coo_frame='body',
    survey="http://voparis-srv-paris.obspm.fr/vo/planeto/hips/CDS_P_Mars_Viking-MDIM21-color/"
)
aladin

Aladin(coo_frame='body', options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_sur…

In [49]:
for mocService in data :
    if not mocService["coverage"].empty() :
        jsonMoc = mocService["coverage"].serialize(format='json', optional_kw_dict=None, pre_v2=False)
        aladin.add_moc_from_dict(jsonMoc, {'opacity' : 0.5, 'name' : mocService["service"]})